In [1]:
import pandas as pd
from pathlib import Path
import sys
sys.path.append("../src")

# Modules to process data
import pull_solar_count_and_capacity as eia
from solar_technical_potential import compute_solar_eligibility
from median_solar_costs import load_and_process_lbnl
from average_electricity_prices import build_eia861_prices
from median_permit_fees import run_pipeline as fees_runpipeline
from median_interconnection_timelines import run_pipeline as ix_runpipeline
from solar_bill_savings import compute_state_bill_savings

# Module to upload data
from drive_uploader import get_drive_service, ensure_path, upload_df_to_drive

KeyboardInterrupt: 

In [ ]:
# Configurations to upload data to Google Drive

# Identify folder ID (the Big Numbers Database)
ROOT_ID = "1DBlVUvspIPTTyZPtVovYtEgUSmQNXmG7"

service = get_drive_service()

In [ ]:
# Load data

# ResStock metadata for solar potential calcs
resstock_metadata = pd.read_csv("../data/resstock_metadata_technical_potential.csv")

# AHJ population data
ahj_pop = pd.read_csv("../data/ahj_distribution_permitting_timelines_pv.csv")[['state', 'name', 'geoid', 'population']]

# Solar TRACE permitting fees
fees = pd.read_csv("../data/solartrace_fees.csv")

# Solar TRACE interconnection timelines
ix = pd.read_csv("../data/solartrace_ix.csv")

# State name <> abbr mapping
name_abbr = pd.read_csv("../data/state_name_abbr.csv")


In [ ]:
# Certain datasets were processed manually based on Ohm data, we will simply load those and upload them without any processing

# AHJ
cancellation_ahj = pd.read_csv("../data/ahj_cancellation_rates.csv")
permitting_timelines_dist_ahj = pd.read_csv("../data/ahj_distribution_permitting_timelines_pv.csv")
permitting_timelines_tech_ahj = pd.read_csv("../data/ahj_median_permitting_timelines_by_tech.csv")
inspection_timelines_ahj = pd.read_csv("../data/ahj_inspection_timelines_pv.csv")

# State
cancellation_state = pd.read_csv("../data/state_cancellation_rates.csv")
permitting_timelines_dist_state = pd.read_csv("../data/state_distribution_permitting_timelines_pv.csv")
permitting_timelines_tech_state = pd.read_csv("../data/state_median_permitting_timelines_by_tech.csv")
inspection_timelines_state = pd.read_csv("../data/state_inspection_timelines_pv.csv")
solar_potential_twh = pd.read_csv("../data/solar_potential.csv")
solar_jobs = pd.read_csv("../data/solar_jobs.csv")


# Upload to Google Drive
folder_id = ensure_path(service, ROOT_ID, ["1. Solar"])

upload_df_to_drive(solar_potential_twh, folder_id, "Potential Solar Generation by State")

folder_id = ensure_path(service, ROOT_ID, ["2. Permitting"])

upload_df_to_drive(cancellation_ahj, folder_id, "Cancellation Rates by AHJ")
upload_df_to_drive(cancellation_state, folder_id, "Cancellation Rates by State")
upload_df_to_drive(permitting_timelines_dist_ahj, folder_id, "Distribution of Solar Permitting Timelines by AHJ")
upload_df_to_drive(permitting_timelines_dist_state, folder_id, "Distribution of Solar Permitting Timelines by State")
upload_df_to_drive(permitting_timelines_tech_ahj, folder_id, "Median Permitting Timeline by Tech by AHJ")
upload_df_to_drive(permitting_timelines_tech_state, folder_id, "Median Permitting Timeline by Tech by State")

folder_id = ensure_path(service, ROOT_ID, ["3. Inspection"])

upload_df_to_drive(inspection_timelines_ahj, folder_id, "Median Inspection Timeline by AHJ")
upload_df_to_drive(inspection_timelines_state, folder_id, "Median Inspection Timeline by State")

folder_id = ensure_path(service, ROOT_ID, ["5. Jobs"])

upload_df_to_drive(solar_jobs, folder_id, "Solar Jobs by State by Category")

'1n_Av0hvKY-Qb2QGhzBJ20CwTZBeIEhDcnarQ7Hi2LnU'

In [ ]:
# Solar/Storage capacity and installations by year by sector

years = range(2017, 2026)

capacity = eia.download_and_aggregate_distributed_solar(years)

# Upload to drive
folder_id = ensure_path(service, ROOT_ID, ["1. Solar"])

upload_df_to_drive(capacity, folder_id, "Solar/Storage Capacity and Installations by State")

'119cdOpazpRnQ0mhCASaGDmSEbi33uPdnBI0bO4LPtuM'

In [ ]:
# Solar eligibility

eligibility = compute_solar_eligibility(resstock_metadata)

# Upload to drive
folder_id = ensure_path(service, ROOT_ID, ["1. Solar"])

upload_df_to_drive(eligibility, folder_id, "Count and Percentage of Solar-eligible Households by State")

'1KvwDc_ZauXgdg3U098wDmO6qHEGIsBbU3STTRvL6pjA'

In [ ]:
# Median solar costs over time

# Google Drive link to the LBNL Tracking the Sun ZIP file
drive_link = "https://drive.google.com/file/d/1NQh4TRC_IqDz2r5vfZuxDm6LGjEuexdu/view"

# FRED API Key
fred_api_key = "2764715428a4687d2a8ce57af948081d"

costs = load_and_process_lbnl(
    drive_url=drive_link,
    fred_api_key=fred_api_key,
    start_year=2000,        # CPI start year
    min_install_year=2000,  # Only installations from 2000 onward
)

# Upload to drive
folder_id = ensure_path(service, ROOT_ID, ["1. Solar"])

upload_df_to_drive(costs, folder_id, "Solar Costs by State Over Time")

Downloading...
From (original): https://drive.google.com/uc?id=1NQh4TRC_IqDz2r5vfZuxDm6LGjEuexdu
From (redirected): https://drive.google.com/uc?id=1NQh4TRC_IqDz2r5vfZuxDm6LGjEuexdu&confirm=t&uuid=4a4829b7-7a49-4db4-824f-d47489ba350f
To: /var/folders/sr/hy8nzjhj7xz25lt74m44nv140000gn/T/lbnl_vjyee8ox/lbnl_latest.zip
100%|██████████| 218M/218M [00:04<00:00, 46.1MB/s] 


Extracting LBNL ZIP …
/var/folders/sr/hy8nzjhj7xz25lt74m44nv140000gn/T/lbnl_vjyee8ox/TTS_LBNL_public_file_29-Sep-2025_all.csv
Loading LBNL CSV: /var/folders/sr/hy8nzjhj7xz25lt74m44nv140000gn/T/lbnl_vjyee8ox/TTS_LBNL_public_file_29-Sep-2025_all.csv


/Users/talorgruenwald/src/permit_power/basic-solar-statistics/notebooks/../src/median_solar_costs.py:255: DtypeWarning: Columns (1,2,3,11,15,16,18,28,29,31,32,34,35,38,39,40,53,54,56,57,59,60,74,75,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(csv_path)


'1f52R9h06Z6d42uWZ1mGmeFBL5RzUlN5g-44AE_eDWV0'

In [ ]:
# Average electricity prices

years = range(2015, 2025)

util_prices, state_prices = build_eia861_prices(years)

# Upload to drive
folder_id = ensure_path(service, ROOT_ID, ["4. Rates"])

upload_df_to_drive(state_prices, folder_id, "Average Electricity Rates by State")
upload_df_to_drive(util_prices, folder_id, "Average Electricity Rates by Utility")

/Users/talorgruenwald/src/permit_power/basic-solar-statistics/notebooks/../src/average_electricity_prices.py:419: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  agg.groupby(["utility_number", "sector"])["price_per_kwh"].pct_change()


'17ct5zKhEzs4VI5m-JQiCQO4hWAOMIKDEsn1bVVb4Dxk'

In [ ]:
# Permitting fees

ahj_df_fees, state_df_fees = fees_runpipeline(fees, ahj_pop, name_abbr)

# Upload to drive
folder_id = ensure_path(service, ROOT_ID, ["2. Permitting"])

upload_df_to_drive(state_df_fees, folder_id, "Permitting Fees by State")
upload_df_to_drive(ahj_df_fees, folder_id, "Permitting Fees by AHJ")

'14rJ8pkxPbQ-HQr9WoTUYcKdnKp3CxdJtmrt_-QcSq4U'

In [ ]:
# Interconnection timelines

ahj_df_ix, state_df_ix = ix_runpipeline(ix)

# Upload to drive
folder_id = ensure_path(service, ROOT_ID, ["3. Interconnection"])

upload_df_to_drive(state_df_ix, folder_id, "Interconnection Timelines by State")
upload_df_to_drive(ahj_df_ix, folder_id, "Interconnection Timelines by AHJ")

'1JQQagE3u7XqEW1Mmx7qJ63uuVX0xVPn8AmUGosxXGHA'

In [ ]:
# Average and lifetime bill savings with solar

base_dir = "/Volumes/Seagate Portabl/permit_power/dgen_runs/per_state_outputs"

savings = compute_state_bill_savings(base_dir, run_name="run_all_states_net_savings_adjust_loan_params")

# Upload to drive
folder_id = ensure_path(service, ROOT_ID, ["1. Solar"])

upload_df_to_drive(savings, folder_id, "Average Annual and Lifetime Solar Savings by State")

'1KNeXwvR13jjojxeKUMks29dAiVfqiyf7vy5Rir7_m40'